# Solve Using QAOA

In [1]:
import pandas as pd
from qiskit.circuit.library import TwoLocal
import numpy as np
import datetime as dt
from qiskit_algorithms import QAOA
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils.algorithm_globals import algorithm_globals

In [2]:
class QuantumSolver:
    df=None #INV.csv
    

    highval = 1000000 # used for G (neglecting some flights)
    inv_id:str
    flight=None
    Q=A=B=N=G=None

    def __init__(self,inv_id,inFile):
        self.df = pd.read_csv(inFile)
        
        startTime=dt.datetime.now()
        self.inv_id=inv_id
        self.lst=self.__preProcess()
        print(len(self.lst),"Total Flights:\n")
        for i in self.lst:
            print(i)
        self.length = len(self.lst)
        self.Q = np.zeros((self.length,self.length))
        self.A,self.B,self.N,self.G=np.zeros_like(self.Q),np.zeros_like(self.Q),np.zeros_like(self.Q),np.zeros_like(self.Q) # all matrices

        print("\nAffected Flight\n")
        print(self.flight)
        ans=self.quantumSolve()
        print(ans)

        print((dt.datetime.now()-startTime))

    
    def __diff(self,date1, time1, date2, time2):
        """
            returns difference in time of date2-date1
        """
        dt1 = dt.datetime.strptime(date1 + " " + time1, "%m/%d/%Y %H:%M")
        dt2 = dt.datetime.strptime(date2 + " " + time2, '%m/%d/%Y %H:%M')
        difference = dt2 - dt1
        return int(difference.total_seconds() // 60)


    def __preProcess(self):

        index=self.df.loc[self.df["InventoryId"]==self.inv_id].index[0]
        flight=self.df.loc[index]
        self.flight=flight
        list_of_feasible_flights=[]

        for i in range(len(self.df)):
            data = self.df.loc[i]
            if index==i:
                continue
            ti = self.__diff(date1=flight["DepartureDate"],time1=flight["DepartureTime"],date2=data["ArrivalDate"],time2=data["ArrivalTime"])
            ti2 = self.__diff(date1=flight["DepartureDate"],time1=flight["DepartureTime"],date2=data["DepartureDate"],time2=data["DepartureTime"])
            # print(ti,ti2)
            print(data["InventoryId"],ti,ti2)
            if ti < 60 or ti2 > 72*60:
                continue
            else:
                list_of_feasible_flights.append(data)
            
        return list_of_feasible_flights

    def __run(self):
        self.graph = dict()

        # Nodes
        # for loop working
        for i in range(len(self.lst)):
            data = self.lst[i]

            if data["InventoryId"] not in self.graph:
                self.graph[data["InventoryId"]] = (
                    data["DepartureAirport"],
                    data["ArrivalAirport"],
                    data["DepartureDate"],
                    data["DepartureTime"],
                    data["ArrivalTime"],
                    data["ArrivalDate"]
                )

                self.Q[i,i] = self.__diff(data["DepartureDate"], data["DepartureTime"], data["ArrivalDate"], data["ArrivalTime"])*100
                if data["DepartureAirport"] == self.flight["DepartureAirport"]:
                    self.A[i, i] = 1
                    self.N[i, i] = 1
                    self.G[i,i]= self.__diff(self.flight["DepartureDate"],self.flight["DepartureTime"],data["DepartureDate"],data["DepartureTime"])*100
                else:
                    self.A[i, i] = 0
                self.B[i, i] = 1 if (data["ArrivalAirport"] == self.flight["ArrivalAirport"]) else 0

            # Edges
                for j in range(len(self.lst)):
                    fl2 = self.lst[j]
                    if self.N[i,i] == 1 and i!=j:
                        self.G[i,j]=0
                    else:
                        if data["InventoryId"] == fl2["InventoryId"]:
                            ti= self.__diff(self.flight["DepartureDate"],self.flight["DepartureTime"],fl2["DepartureDate"],fl2["DepartureTime"])*100
                            self.G[i,j]=ti
                        else:
                            if data["DepartureAirport"] == fl2["ArrivalAirport"]:
                                self.N[j,i] = 1
                                ti = int(self.__diff(fl2["ArrivalDate"],fl2["ArrivalTime"],data["DepartureDate"],data["DepartureTime"])*100 )# minutes(fl2["DepartureTime"]) - minutes(data["ArrivalTime"])
                                if ti <6000 or ti >72000 :
                                    self.G[i,j]=0
                                else:
                                    self.G[i,j]=ti
                            else:
                                self.N[j,i]=0
                                self.G[i,j]=0
        self.G=self.G.astype(int)
        print("Q:\n ",self.Q)
        print("G:\n ",self.G)
        print("A:\n ",self.A)
        print("B:\n ",self.B)
        print("N:\n ",self.N)

    def QAOA_algo(self):
        qp = QuadraticProgram("flights")
        F = self.Q + self.G # quadratic form matrix
        F=F.astype(int)
        print(F)
        qp.minimize(quadratic=F) # matrices fed into the quadratic program
        qp.binary_var_list(self.length)
        qp.linear_constraint(np.ones(self.length,),">",1)
        qp.linear_constraint(self.A.diagonal(),"=",1)
        qp.linear_constraint(self.B.diagonal(),"=",1)
        print(qp.prettyprint())

        qp= QuadraticProgramToQubo().convert(qp)
        # qp.quadratic_constraint(quadratic=self.N,linear=-1*np.ones((self.length,)),sense="=",rhs=0)
        print(qp.prettyprint())

        qubitOp, offset = qp.to_ising()  # conversion into Ising Problem
        
        two = TwoLocal(self.length, 'rx', 'cx', 'linear', reps=2, insert_barriers=True)  # an ansatz circuit
        start=dt.datetime.now()
        algorithm_globals.random_seed = 12345 
        qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
        qaoa_mes.ansatz=two
        qaoa = MinimumEigenOptimizer(qaoa_mes)
        qaoa_result = qaoa.solve(qp)
        print(qaoa_result.prettyprint())
        ans = list(qaoa_result.variables_dict.values())
        print(dt.datetime.now()-start)

        return ans

    def quantumSolve(self) -> list:
        total = []
        self.__run() # we will get the Q A B N G matrices initialised now
        # for i in range(2):    
        ans=self.QAOA_algo()
        total.append(self.__postProcess(ans[:self.length]))
        return total


    def __postProcess(self,bitString:list) -> list:
        flights = []
        for i in range(len(bitString)):
            if bitString[i]==1:
                flights.append(self.lst[i])
        # for i in range(len(flights)):
        #     for j in range(len(flights)):
        #         if i!=j :
        #             self.Q[i,j]=self.highval
        return flights


In [3]:
INVFile=["../INV.csv","../sampleData(toBeDeleted)/INV.csv","../sampleData(toBeDeleted)/INV copy.csv","../sampleData(toBeDeleted)/INV copy 2.csv","../sampleData(toBeDeleted)/INV copy 3.csv","../sampleData(toBeDeleted)/INV copy 4.csv"]

In [4]:
QuantumSolver("INV-ZZ-1875559",INVFile[0])

INV-ZZ-1875551 140 0
INV-ZZ-1875560 442 223
INV-ZZ-1875568 233 203
INV-ZZ-1875569 323 303
INV-ZZ-2847882 1933 1440
INV-ZZ-2048509 10573 10080
INV-ZZ-4381149 129726 130685
INV-ZZ-8169821 131166 132125
INV-ZZ-5260106 123363 122958
INV-ZZ-1682061 -194312 -194405
INV-ZZ-2831256 -192321 -192455
INV-ZZ-2175571 -152311 -151945
INV-ZZ-2361128 -146431 -146144
INV-ZZ-8624303 -114640 -114239
INV-ZZ-2487839 -114031 -112911
INV-ZZ-8710804 -84013 -83963
INV-ZZ-2689731 -83365 -82667
INV-ZZ-3037082 -81859 -81519
INV-ZZ-8761059 -80857 -80526
INV-ZZ-1617666 -77639 -77703
INV-ZZ-8849583 -55831 -55465
INV-ZZ-4896316 -152420 -152436
INV-ZZ-4224955 -130052 -131278
INV-ZZ-4233206 -126271 -125984
INV-ZZ-3665854 -140647 -140148
INV-ZZ-1005162 -138723 -139715
INV-ZZ-2694616 -222145 -222268
INV-ZZ-3238963 -220450 -219170
INV-ZZ-4607320 -218595 -218344
INV-ZZ-3715300 -214950 -215092
INV-ZZ-7580733 -213413 -214696
INV-ZZ-5409981 -237032 -237470
INV-ZZ-8022558 -235988 -236144
INV-ZZ-7150252 -232309 -232934
INV-ZZ-9

objective function value: 46300.0
variable values: x0=1.0, x1=0.0, x2=0.0, x3=1.0, x4=0.0, c0@int_slack@0=1.0, c0@int_slack@1=0.0, c0@int_slack@2=0.0
status: SUCCESS
0:00:01.329354
[[InventoryId         INV-ZZ-1875551
ScheduleId          SCH-ZZ-7283355
AircraftType            Boeing 787
DepartureDate           05/25/2024
ArrivalDate             05/25/2024
DepartureAirport               CNN
ArrivalAirport                 NNR
DepartureTime                 4:37
ArrivalTime                   6:57
Name: 0, dtype: object, InventoryId         INV-ZZ-1875569
ScheduleId          SCH-ZZ-7283355
AircraftType            Boeing 787
DepartureDate           05/25/2024
ArrivalDate             05/25/2024
DepartureAirport               CAD
ArrivalAirport                 MAA
DepartureTime                 9:40
ArrivalTime                  10:00
Name: 3, dtype: object]]
0:00:01.466589
